In [3]:
import networkx as nx
import matplotlib.pyplot as plt
import urllib.request
import gzip
from collections import defaultdict, Counter
import pandas as pd
import csv
import requests
import itertools
import numpy as np
import json
import random as rd
import math
import pickle as pk

import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

### Network Analysis and Visualization

Welcome to the "Network Analysis and Visualization" notebook!

In this notebook, we will be introducing basic and advanced layout options for visualizing networks effectively. 
Another way to improve visualization is to color the nodes in an informative way; in this notebook we will also introduce how to color nodes by communities and node annotations.

The following key topics will be covered:

1. Generate a network from expression profile data, perform edge filtering and run a basic layout algorithm to quickly produce content for VR.

2. Community Detection and Node Coloring: Communities are densely interconnected groups of nodes within a network. We will use a well-known algorithm to assign distinct colors to nodes belonging to the same community. This visualization enhancement helps identify cohesive groups and their relationships within the network.

3. Advanced Network Layouts: While the traditional force-directed layout (such as the Fruchterman-Reingold algorithm) is commonly used, we will introduce additional layout options that offer different visual perspectives of the network. These layouts can reveal hidden patterns, optimize node placement, or emphasize specific structural characteristics.





### Human Protein Atlas data

The Human Protein Atlas is a comprehensive resouce mapping proteins in cells, tissues, and organs through the whole range of -omics data.

Today we will use the "Tissue" section which provides the protein expression profiles in 44 different human tissues based on immunohistochemistry data.

The tab separated file includes several columns:
1. **Gene** -- the Ensembl gene identifier
2. **Tissue** -- the name of the tissue
3. **Cell type** -- the annotated cell type.
4. **Level** -- the expression level
5. **Reliability** -- the confidence of value in the "Level" column.

The data is based on The Human Protein Atlas version 22.0 and Ensembl version 103.38.

(source file 'normal_tissue.tsv' can be downloaded here: `www.proteinatlas.org/about/download`)

In [2]:
# Define the URL path where the file is located
path = 'https://www.proteinatlas.org/download/'

# Download and convert the data file into a DataFrame
data = pd.read_csv(path + 'normal_tissue.tsv.zip', sep='\t', compression='zip')
# Extract the 'Gene name' column values as gene symbols
gene_symbols = data['Gene name'].values

# Print the number of rows in the gene_symbols array
print('#total rows:', len(gene_symbols))

# Filter out rows where 'Reliability' is 'Uncertain'
data = data[data['Reliability'] != 'Uncertain']
# Define the values to exclude from the 'Level' column
exclude_values = [float('nan'), 'Not representative', 'Not detected']
# Filter out rows with excluded values in the 'Level' column
data = data[~data['Level'].isin(exclude_values)]


# Update variables with filtered column values
gene_symbols = data['Gene name'].values
tissue = data['Tissue'].values
celltype = data['Cell type'].values
explevel = data['Level'].values
reliab = data['Reliability'].values

# Print the number of rows after filtering and the count of unique tissues and cell types
print('# rows after filtering:', len(gene_symbols))
print('There are %s different tissues and %s different cell types remaining after filtering.' %
      (len(set(tissue)), len(set(celltype))))

# Create a dictionary to connect gene names with cell types
gene_celltype_dict = data.groupby('Gene name')['Cell type'].apply(list).to_dict()
# Create a dictionary where each gene name is associated with a set of unique cell types
gene_celltypeunique_dict = {k: set(v) for k, v in gene_celltype_dict.items()}

# Create a dictionary to connect cell types with genes
celltype_genes_dict = defaultdict(list)
for gene, celltypes in gene_celltypeunique_dict.items():
    for ct in celltypes:
        celltype_genes_dict[ct].append(gene)

# Create a dictionary where each cell type is associated with a set of unique genes
celltype_genesunique_dict = {k: set(v) for k, v in celltype_genes_dict.items()}
# Create a dictionary to connect cell types with tissues
celltype_tissue_dict = data.groupby('Cell type')['Tissue'].apply(list).to_dict()
# Create a dictionary where each cell type is associated with a set of unique tissues
celltype_tissueunique_dict = {k: set(v) for k, v in celltype_tissue_dict.items()}


# rows:  1197500
# rows after filtering:  517598
There are 63 different tissues and 143 different cell types remaining after filtering.


### Functions for preprocessing 

In [13]:
def jaccard_index(set1, set2):
    """
    Calculates the Jaccard index between two sets.
    The Jaccard index measures the similarity between two sets of genes
    by dividing the size of their intersection by the size of their union.

    Returns:
        float: The Jaccard index value, which represents the similarity between the two sets.
    """

    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    jaccard_index = intersection / union if union != 0 else 0
    return jaccard_index


def remove_low_ranked_edges(network, ranking_dict):
    """
    Filter procedure 
    Filters edges of a given network based on a dictionary 
    (ranking_dict) which contains edges with a score/weight
    The edge-removal stops at the breaking point of the network 
    -> number of components > 1. 

    Returns:
        nx.Graph: new network with a minimal set of edges
    """

    # Sort edges based on the ranking score in ascending order
    sorted_edges = sorted(ranking_dict.items(), key=lambda x: x[1])

    # Create a copy of the original network
    updated_network = network.copy()

    # Iterate through the sorted edges and remove low-ranked edges until the network becomes disconnected
    for edge, _ in sorted_edges:
        updated_network.remove_edge(*edge)  # Remove the edge from the network

        # Check if the network becomes disconnected
        if not nx.is_connected(updated_network):
            updated_network.add_edge(*edge)  # Add the edge back
        else:
            network = updated_network.copy()  # Update the network

    return updated_network


def make_json(name, network, positions, node_color = '#40b9d4', link_color = '#999999', annotations = 'None', communities = 'None'): 
    """
    Generates a JSON file from a given network graph using the specified parameters.

    Args:
        name (str, optional): Name of the graph.
        network (networkx.Graph): Network graph object.
        positions (dict): Dictionary mapping node IDs to their positions.
        node_color (dict): Dictionary mapping node IDs to their (hex-)colors.
        link_color (str or dict): (Hex-)color value for all links in the graph or dict with node tuple as key and hex color as value.
        communities (dict): 'None' for no communities (default) or dictionary mapping node IDs to their corresponding community ID.  
        annotations (dict): Dictionary mapping node IDs to a list of annotations.

    Returns:
        None

    """

    # --------------------------
    # Generate VR GRAPH 
    # --------------------------
    GVR = nx.Graph()
    GVR.graph['name'] = name

    # --------------------------------------
    # LOOKUP FOR NODE NAMES INTO IDs and vv
    # --------------------------------------
    d_idx_node = {}
    d_node_idx = {}
    for i, node in enumerate(sorted(network.nodes())):
        d_idx_node[i] = node
        d_node_idx[node] = i
    GVR.add_nodes_from(d_idx_node.keys())

    for edge in network.edges()(data=True):
        GVR.add_edge(d_node_idx[edge[0]],d_node_idx[edge[1]])

    # --------------------------
    # POS 
    # --------------------------
    if isinstance(positions[next(iter(positions))], list):
        pass
    else:
        for key in positions:
            positions[key] = positions[key].tolist()

    posG = {d_node_idx[node]: list(xyz) for node, xyz in positions.items()}
    nx.set_node_attributes(GVR, posG, name="pos")

    # # --------------------------
    # # CLUSTER 
    # # --------------------------
    if communities == 'None':
        d_VRids_cluster = dict(zip(d_idx_node.keys(), [0 for _ in d_idx_node.keys()]))
    else:
        d_VRids_cluster = {d_node_idx[node]: str(cl_id) for node, cl_id in communities.items()}        
    nx.set_node_attributes(GVR, d_VRids_cluster, name="cluster")


    # --------------------------
    # NODE COLOR
    # --------------------------
    d_node_colors={}

    if isinstance(node_color, dict):
        for nodeid in GVR.nodes():
            d_node_colors[nodeid] = node_color[d_idx_node[nodeid]] 
    else:
        for nodeid in GVR.nodes():
            d_node_colors[nodeid] = node_color

    nx.set_node_attributes(GVR, d_node_colors, name="nodecolor")

    # --------------------------
    # LINK COLOR
    # --------------------------
    if isinstance(link_color, dict):
        # for different link colors
        d_edge_color = {}
        for a,b in GVR.edges():
            try:
                color = link_color[(d_idx_node[a],d_idx_node[b])]
            except KeyError: 
                color = link_color[(d_idx_node[b],d_idx_node[a])]
            d_edge_color[(a,b)] = color
    else:
        # for unique link colors
        d_edge_color = {}
        for a,b in GVR.edges():
            d_edge_color[(a,b)] = link_color

    nx.set_edge_attributes(GVR, d_edge_color, name="linkcolor")

    # --------------------------
    # NODE ANNOTATION
    # --------------------------
    if isinstance(annotations, dict):

        l_annotations = [[str(d_idx_node[nodeid])] + [ annotation for annotation in annotations[d_idx_node[nodeid]]] for nodeid in sorted(GVR.nodes())]
        d_annotations = dict(zip(sorted(GVR.nodes()), l_annotations))
    else:
        d_annotations = {nodeid: [str(d_idx_node[nodeid])] for nodeid in GVR.nodes()}

    nx.set_node_attributes(GVR, d_annotations, name="annotation")

    # --------------------------
    # MAKE JSON fo uploader
    # --------------------------

    G_json = json.dumps(nx.node_link_data(GVR))

    with open(GVR.name+".json", "w") as outfile:
        outfile.write(G_json)



def plotly_preview(G, pos, node_colors=None, edge_colors=None):

    """
    Generate a 3D network visualization using Plotly.

    Parameters:
        - G (networkx.Graph): The graph object representing the network.
        - pos3D (dict): A dictionary mapping each node to its 3D coordinates (x, y, z).
        - node_colors (dict, optional): A dictionary mapping nodes to custom hex colors.
                                        If not provided, the default color is '#40b9d4'.
        - edge_colors (dict, optional): A dictionary mapping edges to custom hex colors.
                                        If not provided, the default color is 'gray'.

    Output:
        - An HTML file named 'network_visualization.html' is generated, which opens in a web browser.

    Example usage:
        node_colors = {1: '#ff0000', 2: '#00ff00', 3: '#0000ff'}
        edge_colors = {(1, 2): '#ff00ff', (2, 3): '#ffff00'}
        pos = nx.spring_layout(G,dim=3)
        plotly_review_2(G, pos, node_colors, edge_colors)
    """

    # Create a Plotly figure
    fig = go.Figure()

    # Add nodes to the figure
    for node in G.nodes():
        x, y, z = pos[node]
        color = node_colors[node] if node_colors and node in node_colors else '#40b9d4'
        fig.add_trace(go.Scatter3d(
            x=[x],
            y=[y],
            z=[z],
            mode='markers',
            marker=dict(
                size=5,
                color=color,
            ),
            name=str(node),
            text=str(node),
            hovertemplate=None,
        ))

    # Add edges to the figure
    for edge in G.edges():
        x0, y0, z0 = pos[edge[0]]
        x1, y1, z1 = pos[edge[1]]
        edge_color = edge_colors[edge] if edge_colors and edge in edge_colors else 'gray'
        fig.add_trace(go.Scatter3d(
            x=[x0, x1],
            y=[y0, y1],
            z=[z0, z1],
            mode='lines',
            line=dict(
                color=edge_color,
                width=1,
            ),
            hoverinfo='none',
        ))

    # Set layout options
    fig.update_layout(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
        ),
        showlegend=False,
        hovermode='closest',
        margin=dict(l=0, r=0, b=0, t=0),
    )

    # # Save the figure as an HTML file
    # fig.write_html('network_preview.html', auto_open=True)

    # Display the plot inline in the notebook
    iplot(fig)

### Cell type expression similarity network

Let's build a network where the **nodes** are cell types (ex. neuronal cells, endothelian cells, macrophages, ...) and **links** represent the similarity of genes expressed between two cell types.

We use the Jaccard index to assign a number to the similarity between two sets of genes.  The Jaccard index will range from [0,1], where 0 means that there aren't any shared genes, and 1 means all the genes are shared.

Later, we can choose a cut-off value to filter edges.  While we can choose a cut off arbitrarily, another way could be by inspecting the distribution of Jaccard indices.

At this point, we will just generate the network using all available edges, each with its respective similarity value (no cutoff).

In [14]:
pairwise_celltype_combinations = list(itertools.combinations(celltype_genesunique_dict.keys(), 2))
sim_cutoff = 0.0

G_celltype = nx.Graph()
l_sim_scores = []
for t1, t2 in pairwise_celltype_combinations:
    sim_score = jaccard_index(celltype_genesunique_dict[t1],celltype_genesunique_dict[t2])
    l_sim_scores.append(sim_score)
    if sim_score > sim_cutoff:
        G_celltype.add_edge(t1,t2,weight = sim_score)

print('# nodes: ', G_celltype.number_of_nodes())
print('# edges: ', G_celltype.number_of_edges())
print('# of connected components: ', nx.number_connected_components(G_celltype))
print('network density (# present links divided by all possible links): ', nx.density(G_celltype))


# nodes:  143
# edges:  7948
# of connected components:  1
network density (# present links divided by all possible links):  0.782822810991825


### Filtering edges

It's often difficult to gain insight from a network that is too dense -- everything is just connected to everything!  In this case, it might be useful to filter some less influencial edges.

One possible way to filter edges is based on edge centrality measures.  In the following block we demonstrate this using the betweenness cetnrality measure.

https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.edge_betweenness_centrality.html

For more filtering techniques please refer to the following notebook: `protein_atlas_celltypenetwork_edgefiltering.ipynb`

In [40]:
d_edge_btw = dict(nx.edge_betweenness_centrality(G_celltype, weight=sim_score))

# optional: look at the betweenness distribution
# plt.figure(figsize=(5,4))
# plt.hist(d_edge_btw.values(), bins='auto', alpha=0.7, rwidth=0.85)
# plt.grid(axis='y', alpha=0.75)
# plt.xlabel('edge betweeness')
# plt.ylabel('Frequency')
# plt.xscale('log')


The next filtering step may take a minute or two to compute...

In [9]:
G_celltype_filtered = remove_low_ranked_edges(G_celltype, d_edge_btw)

print('# nodes: ', G_celltype_filtered.number_of_nodes())
print('# edges: ', G_celltype_filtered.number_of_edges())
print('# of connected components: ', nx.number_connected_components(G_celltype_filtered))
print('network density (# present links divided by all possible links): ', nx.density(G_celltype_filtered))

# nodes:  143
# edges:  142
# of connected components:  1
network density (# present links divided by all possible links):  0.013986013986013986


Since generating the filtered version of the network requires some time, it would be advisable to save the result as a pickle for future use.

In [15]:
### OPTIONAL ###

# Pickle the object
# with open('G_celltype_filtered.pkl', 'wb') as file:
#     pk.dump(G_celltype_filtered, file)

# # Load the pickled object
with open('G_celltype_filtered.pkl', 'rb') as file:
    G_celltype_filtered = pk.load(file)

print('# nodes: ', G_celltype_filtered.number_of_nodes())
print('# edges: ', G_celltype_filtered.number_of_edges())
print('# of connected components: ', nx.number_connected_components(G_celltype_filtered))
print('network density (# present links divided by all possible links): ', nx.density(G_celltype_filtered))


# nodes:  143
# edges:  142
# of connected components:  1
network density (# present links divided by all possible links):  0.013986013986013986


Now we have our network!  The next step for visualizing a network is to assign some positions for the nodes, aka creating a layout.

### Basic layout

There are a lot of different layout tools.  We will use the ```spring_layout``` which is of the most popular layout method.  

However, this approach has some limitations, especially for larger networks -- it often results in a not very helpful "hairball".  We'll show you some alternative layouts later!

In [16]:
pos3D_rnd = nx.random_layout(G_celltype_filtered, dim=3)

pos3D_spg = nx.spring_layout(G_celltype_filtered, dim=3, iterations=200)

plotly_preview(G_celltype_filtered, pos = pos3D_spg)

### Node annotations

The VR platform requires the user to provide the networkx object and the 3D position of nodes.  However, it might also be useful to annotate nodes (i.e. label them).  These annotations can be displayed on the network or searched for from the VR interface.

Now we'll annotate each node (a cell type) with the list of tissues in which this cell type appears.  Earlier when importing the data, we created a dictionary ```celltype_tissueunique_dict``` that contains the celltypes (nodes) as keys and the tissues as values.  We can hand the dictionary along with the networkx object and node positions to the ```make_json``` function.

In [17]:

# JSON files required for the VR uploader
# make random layout
make_json(name = '01_celltype_rnd', network=G_celltype_filtered, positions=pos3D_rnd, annotations = celltype_tissueunique_dict)
# make spring layout
make_json(name = '02_celltype_spg', network=G_celltype_filtered, positions=pos3D_spg, annotations = celltype_tissueunique_dict)


### Communities
Another thing we can do to improve our network visualization is to color the nodes in an informative way.  Here we will find communities in the network and color the network by their community membership.

Communities (loosely speaking) are collections of nodes that are more densely connected with each other than the rest of the network.

We'll use a modularity approach to demonstrate the procedure.  For the official documentation check out the following link:

[https://python-louvain.readthedocs.io/en/latest/api.html](https://python-louvain.readthedocs.io/en/latest/api.html)

In [18]:
# ! pip install python-louvain
from community import community_louvain
import matplotlib.colors as mcolors

random_seed = 42
partition = community_louvain.best_partition(G_celltype_filtered, random_state=random_seed)
n_coms = len(set(partition.values()))

# generate colors
cmap = plt.get_cmap('tab10')  # Choose a colormap (e.g., 'tab10', 'Set3', 'Dark2', etc.)
colors = [mcolors.to_hex(cmap(i)) for i in range(n_coms)]

#  dictionary to assign colors to nodes
d_node_color = {node: colors[comm] for node, comm in partition.items()}


In [19]:
plotly_preview(G_celltype_filtered, pos = pos3D_spg, node_colors = d_node_color)

The following cluster labels have been generated by ChatGPT for illustrative purposes

In [20]:
d_cluster_labels = { 0: 'Epithelial and Neural Cells', 1: 'Tissues and Nerves',2: 'Tissues and Secretory Systems',
                        3: 'Glands, Nervous Tissue, and Supportive Structures.',4: 'Epithelial and Supportive Cells',
                        5: 'Connective and Tubular Structures.',6: 'Glands, Tissues, and Reproductive Systems',
                        7: 'Glands, Tubules, and Neural Structures',8: 'Various Tissues and Functional Regions',
                        9: 'Epithelial and Connective Tissues.',10: 'Processes in Neural Tissues',
                        11: 'Neurological, Reproductive, and Exocrine Systems',12: 'Nervous, Reproductive, and Supporting Tissues',
                        13: 'Diverse Specialized Cells'}

communities_with_labels_dict = {}
for node, clid in partition.items():
    communities_with_labels_dict[node] = d_cluster_labels[clid]

Two new arguments can now be given to the `make_json`

1. different colors for nodes according to communities

2. community labels that can be seen in VR


In [21]:

make_json(name = '03_celltype_spg_communs', network=G_celltype_filtered, positions=pos3D_spg, annotations = celltype_tissueunique_dict, node_color = d_node_color, communities = communities_with_labels_dict)

For larger networks with a greater number of edges, applying edge colors within a community can enhance the visual appeal of the visualization.

In [22]:
edge_colors = {}
# Iterate over edges and assign colors
for edge in G_celltype.edges():
    source, target = edge
    if partition[source] == partition[target]:
        edge_colors[edge] = d_node_color[source]  # or node_colors[target]
    else: 
        edge_colors[edge] = '#aaaaaa'


plotly_preview(G_celltype_filtered,pos3D_spg,node_colors = d_node_color,edge_colors = edge_colors)

In [23]:
make_json(name = '04_celltype_spg_communs', network=G_celltype_filtered, positions=pos3D_spg, annotations = celltype_tissueunique_dict, node_color = d_node_color,link_color=edge_colors, communities = communities_with_labels_dict)

### Advanced layouts

As an alternative to ```spring_layout``` we will go through some more use cases from this paper:

[Cartographs enable interpretation of complex network visualizations](https://www.nature.com/articles/s43588-022-00203-6)
Nature Computational Science **2**, 76–77 (2022)

We will be experimenting with two CartoGraph-layouts:

1. **global**

The "global" layout employs a random walk with restart algorithm and calculates visiting probabilities for each node, which then serve as features. The feature vectors are compared pairwise using cosine similarity and projected into a latent 3D space using UMAP.

2. **importance**

The "importance" layout calculates four features: degree, closeness, betweenness, and eigenvector centrality.
These features are further reduced to three dimensions using UMAP.
(For this method you could also use more or different network features)


In [10]:
# ! pip install cartoGRAPHs
# ! pip install pymysql

from cartoGRAPHs import *

In [24]:

posG3D_global = generate_layout(G_celltype_filtered, 
                        dim = 3,
                        layoutmethod = 'global',
                        dimred_method='umap'
                        )

plotly_preview(G_celltype_filtered,posG3D_global,node_colors = d_node_color)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/umap/spectral.py:260: UserWarning:

failed. This is likely due to too small an eigengap. Consider
adding some noise or jitter to your data.

Falling back to random initialisation!



In certain layouts or small networks, it is possible for points to be located very close to each other, resulting in clumps. 

To mitigate this, we can introduce a small amount of noise to the coordinates, which helps disentangle their positions.

However, it is important to experiment with the displacement factor cautiously to ensure that the underlying network structure is not inadvertently compromised or distorted.

In [25]:
def adjust_point_positions(points, displacement_factor):
    return {key: [coord + rd.gauss(0, displacement_factor) for coord in coords]
            for key, coords in points.items()}

posG3D_global_jitter = adjust_point_positions(posG3D_global, 3e-2 )

plotly_preview(G_celltype_filtered,posG3D_global_jitter,node_colors = d_node_color)

make_json(name = '04_celltype_global', network=G_celltype_filtered, positions=posG3D_global_jitter, node_color = d_node_color, annotations = celltype_tissueunique_dict, communities = partition)

In [26]:
posG3D_importance = generate_layout(G_celltype_filtered, 
                        dim = 3, 
                        layoutmethod = 'importance',
                        dimred_method='umap'
                        )

# add some noise
posG3D_importance_jitter = adjust_point_positions(posG3D_importance, 2e-2 )
plotly_preview(G_celltype_filtered,posG3D_importance_jitter,node_colors = d_node_color)
make_json(name = '05_celltype_importance', network=G_celltype_filtered, positions=posG3D_importance_jitter, node_color = d_node_color, annotations = celltype_tissueunique_dict, communities = partition)

### Use z-axis for annotation data 

In the next step only the x- and y-axes will be used to represent the structural information of a network displayed through a spring layout. 

The z-axis, however, is available for other purposes. In this case, we want to utilize the z-axis to indicate the number of genes associated with a specific cell type.

In [27]:
# Make 2D spring layout and set z variable to zero

pos2D_spg = nx.spring_layout(G_celltype_filtered, dim=2, iterations=200)

pos3D_flatspg = {}
for k, value in pos2D_spg.items():
    new_value = np.zeros(3) 
    new_value[:2] = value    
    pos3D_flatspg[k] = new_value

plotly_preview(G_celltype_filtered,pos3D_flatspg,node_colors = d_node_color)
make_json(name = '06_celltype_flatspg', network=G_celltype_filtered, positions=pos3D_flatspg, node_color = d_node_color, annotations = celltype_tissueunique_dict, communities = partition)

In [28]:
# Give the z-axes a custom value (here the number of associated tissues)
pos3D_customz = {}
for k, value in pos2D_spg.items():
    new_value = np.zeros(3)  
    new_value[:2] = value    
    new_value[2] = len(celltype_tissueunique_dict[k])
    pos3D_customz[k] = new_value

plotly_preview(G_celltype_filtered,pos3D_customz,node_colors = d_node_color)
make_json(name = '07_celltype_customz', network=G_celltype_filtered, positions=pos3D_customz, node_color = d_node_color, link_color = '#aaaaaa', annotations = celltype_tissueunique_dict, communities = partition)